### Imports ###

In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time

### Running on GPU ###

In [45]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


### Hyperparameters ###

In [46]:
BATCH_SIZE = 128

### Dataset ###

In [47]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train) 
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2) 

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


### Réseau neuronal original

In [48]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def count_maxpool_operations(self, input, output, pool_layer, out_channels):
        kernel_maxpooling = pool_layer.kernel_size
        stride = pool_layer.stride
        padding = pool_layer.padding
        output_height = output.shape[2]
        output_width = output.shape[3]
        out_channels =  output.shape[1]
        num_max = output_height * output_width * (kernel_maxpooling**2 -1) * out_channels
        return num_max

    def count_conv_operations(self, input, output, output_pooled, conv_layer, pool_layer):
        out_channels, in_channels = output.size(1), conv_layer.in_channels
        output_height, output_width = output.size(2), output.size(3)
        filter_size = conv_layer.kernel_size[0]
        stride = conv_layer.stride[0]
        padding = conv_layer.padding[0]
        num_mults = output_height * output_width * in_channels * filter_size ** 2 * out_channels
        num_adds = output_height * output_width * in_channels * filter_size ** 2 * out_channels
        num_maxs = self.count_maxpool_operations(output, output_pooled, pool_layer, out_channels)
        total_ops = num_mults + num_adds + num_maxs
        return num_mults, num_adds, num_maxs, total_ops

    def count_operations(self, x):
        conv1_out = self.conv1(x)
        conv1_out_pooled = self.pool(F.relu(conv1_out))
        conv2_out = self.conv2(conv1_out_pooled)
        conv2_out_pooled = self.pool(F.relu(conv2_out))
        conv1_ops = self.count_conv_operations(x, conv1_out, conv1_out_pooled, self.conv1, self.pool)
        conv2_ops = self.count_conv_operations(conv1_out_pooled, conv2_out, conv2_out_pooled, self.conv2, self.pool)
        return conv1_ops, conv2_ops        

    def count_fc_operations(self, input, fc_layer):
        in_features = fc_layer.in_features
        out_features = fc_layer.out_features
        num_mults = out_features * in_features
        num_adds = out_features * in_features
        num_maxs = 0    
        total_ops = num_mults + num_adds
        return num_mults, num_adds, num_maxs, total_ops

    def count_total_operations(self, x):
        conv1_ops, conv2_ops = self.count_operations(x)
        fc1_ops = self.count_fc_operations(x, self.fc1)
        fc2_ops = self.count_fc_operations(x, self.fc2)
        fc3_ops = self.count_fc_operations(x, self.fc3)
        total_ops = sum(op[3] for op in [conv1_ops, conv2_ops, fc1_ops, fc2_ops, fc3_ops])
        return total_ops


### Dans la partie entraînement du réseau CNN, lister les différentes couches et sous-couches :
1. **Deux couches de convolution :**
   - `self.conv1`: Première couche de convolution avec 3 canaux d'entrée et 6 canaux de sortie, utilisant un noyau de taille 5x5.
   - `self.conv2`: Deuxième couche de convolution avec 6 canaux d'entrée et 16 canaux de sortie, également avec un noyau de taille 5x5.

   - **Deux sous-couche non linéaire :** Après chaque opération de convolution, une activation ReLU est appliquée.

2. **Deux couches de max pooling :**

   - `self.pool`: Utilisé après chaque couche de convolution pour réduire la dimensionnalité de la sortie.

3. **Trois couches entièrement connectées :**
   - `self.fc1`: Première couche entièrement connectée avec 16*5*5 entrées et 120 sorties.
   - `self.fc2`: Deuxième couche entièrement connectée avec 120 entrées et 84 sorties.
   - `self.fc3`: Troisième couche entièrement connectée avec 84 entrées et 10 sorties.

   - **Trois sous-couche linéaire :** Chaque couche entièrement connectée effectue une transformation linéaire des caractéristiques d'entrée.
   - **Deux sous-couche non linéaire :** Après chaque transformation linéaire, une activation ReLU est appliquée, introduisant de la non-linéarité.
  
4. **Quarte sous-couches Relu**

In [49]:
net = Net().to(device)
for p in net.parameters(): print(p.size())

torch.Size([6, 3, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


### Donner la taille des différents tenseurs de données Xn et de poids Wn le long du calcul. ###

1. **Convolutional Layer 1 (conv1):**
- X1 = [3, 32, 32]
- Poids W1:
    - Poids de convolution: torch.Size([6, 3, 5, 5])
    - Bias: torch.Size([6])

2. **Convolutional Layer 2 (conv2):**
    - X2 = [6, 14, 14]
- Poids W2:
    - Poids de convolution: torch.Size([16, 6, 5, 5])
    - Bias: torch.Size([16])

3. **Fully Connected Layer 1 (fc1):**
- X3: [400]
- Poids W3:
    - Poids de linéaire: torch.Size([120, 400])
    - Bias: torch.Size([120])

4. **Fully Connected Layer 2 (fc2):**
- X4: [120]
- Poids W4:
    - Poids de linéaire: torch.Size([84, 120])
    - Bias: torch.Size([84])
  
5. **Fully Connected Layer 3 (fc3):**
- X5: [84]
- Poids W5:
    - Poids de linéaire: torch.Size([10, 84])
    - Bias: torch.Size([10])

## Model Residual Net ##

In [50]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(outchannel)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(outchannel)
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.shortcut(x)
        out = self.relu(out)
        return out

In [51]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 64, num_blocks[0], stride=1)
        self.fc = nn.Linear(1024, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        inchannel = 64 
        for stride in strides:
            layers.append(block(inchannel, channels, stride))
            inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = F.avg_pool2d(out, 8)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out

    def count_conv_operations(self, input, output, output_pooled, stride, padding, kernel_size):
        out_channels = output.size(1)
        output_height, output_width = output.size(2), output.size(3)
        num_mults = output_height * output_width * input.size(1) * kernel_size ** 2 * out_channels
        num_adds = output_height * output_width * input.size(1) * kernel_size ** 2 * out_channels
        num_maxs = self.count_maxpool_operations(output, output_pooled)
        total_ops = num_mults + num_adds + num_maxs
        return num_mults, num_adds, num_maxs, total_ops

    def count_maxpool_operations(self, output, output_pooled):
        kernel_size = 2 
        stride = 2 
        padding = 0
        output_height = output.shape[2]
        output_width = output.shape[3]
        out_channels = output.shape[1]
        num_max = ((output_height + 2 * padding - kernel_size) // stride + 1) * ((output_width + 2 * padding - kernel_size) // stride + 1) * (kernel_size ** 2 - 1) * out_channels
        return num_max

    def count_fc_operations(self, input, fc_layer):
        in_features = fc_layer.in_features
        out_features = fc_layer.out_features    
        num_mults = out_features * in_features
        num_adds = out_features * in_features
        num_maxs = 0
        total_ops = num_mults + num_adds
        return num_mults, num_adds, num_maxs, total_ops

    def count_total_operations(self, x):
        conv1_out = self.conv1(x)
        conv1_out_pooled = self.relu(self.bn1(conv1_out))
        conv1_ops = self.count_conv_operations(x, conv1_out, conv1_out_pooled, self.conv1, None, kernel_size=3)

        layer1_out = self.layer1(conv1_out_pooled)
        layer1_out_pooled = F.avg_pool2d(layer1_out, 8)
        layer1_ops = self.count_conv_operations(conv1_out_pooled, layer1_out, layer1_out_pooled, stride=1, padding=1, kernel_size=3)

        fc_ops = self.count_fc_operations(x, self.fc)

        total_ops = sum(op[3] for op in [conv1_ops, layer1_ops, fc_ops])
        return total_ops

In [52]:
def ResNet18():
    return ResNet(ResidualBlock, [2, 2])

**1. Le nombre de couches et sous-couches de Residual Net:**
- 9 couches de convolution: Dans ce cas **[2, 2]**, la fonction ResNet18 renvoie un modèle ResNet comprenant des blocs résiduels de type ResidualBlock et deux étapes, chaque étape ayant 2 blocs résiduels. Par conséquent, il y a un total de 4 blocs résiduels. Chaque bloc résiduel contient 2 couches de convolution. De plus, dans le modèle ResNet, il y a une couche de convolution initiale `self.conv1`. Ainsi, 1 + 4 * 2 = 9 couches de convolution.
  
- 9 couches de Batch Normalization: 
Chaque bloc résiduel contient deux couches de Batch Normalization, donc avec quatre blocs résiduels, il y a au total 2 * 4 = 8 couches de Batch Normalization. Ajoutant la couche de Batch Normalization utilisée après la première couche de convolution, cela donne un total de 8 + 1 = 9 couches de Batch Normalization

- 8 couches ReLU: Chaque bloc résiduel contient 2 sous-couches ReLU, une après chaque opération de convolution. Comme il y a 4 blocs résiduels dans ResNet18 **[2, 2]**, cela fait un total de 4 * 2 = 8 sous-couches ReLU.
  
- 1 couche entièrement connectée
- 1 sous-couche de Average Pooling

In [53]:
netRes = ResNet18().to(device)
for p in netRes.parameters(): print(p.size())

torch.Size([64, 3, 3, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64, 64, 3, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64, 64, 3, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64, 64, 3, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64, 64, 3, 3])
torch.Size([64])
torch.Size([64])
torch.Size([10, 1024])
torch.Size([10])


**2. identification la taille des différents tenseurs de données Xn et de poids Wn le long du calcul:**

**Données d'entrée :**
- X0: [3, 32, 32]

**Couche de convolution 1 (conv1) :**
- Données de sortie (X1) : Taille [64, 32, 32]
- Poids du filtre (W1) : Taille **[64, 3, 3, 3]**
- Biais : Taille [64]

**Couche de normalisation par lots 1 :**
- Données de sortie (X2) : Taille [64, 32, 32]

**Bloc résiduel 1 :**
- Données de sortie (X3) : Taille [64, 32, 32]
- Poids du premier filtre (W2) : Taille **[64, 64, 3, 3]**
- Biais du premier filtre: Taille [64]
- Poids du deuxième filtre (W3) : Taille **[64, 64, 3, 3]**
- Biais du deuxième filtre: Taille [64]

**Bloc résiduel 2 :**
- Données de sortie (X4) : Taille [64, 32, 32]
- Poids du premier filtre (W4) : Taille **[64, 64, 3, 3]**
- Biais du premier filtre: Taille [64]
- Poids du deuxième filtre (W5) : Taille **[64, 64, 3, 3]**
- Biais du deuxième filtre: Taille [64]

**Couche de pooling moyenne :**
- Données de sortie (X5) : Taille [64, 4, 4]

**Couche d'aplatissement :**
- Données de sortie (X6) : Taille [1024]

**Couche entièrement connectée (fc) :**
- Données de sortie (X7) : Taille [10]
- Poids du filtre (W7) : Taille **[10, 1024]**
- Biais: Taille [10]

**3. identification de toutes les fonctions successives (les “Fn”) avec leurs types:**

**Couche de convolution 1 (conv1) :**
- Type de fonction : Convolution nn.Conv2d
- Fonction d'activation : nn.ReLU
- Fonction de normalisation : nn.BatchNorm2d

**Blocs résiduels (layer1) :**
- Type de fonction : Bloc résiduel personnalisé ResidualBlock
- Fonction de convolution : nn.Conv2d
- Fonction d'activation : nn.ReLU
- Fonction de normalisation : nn.BatchNorm2d

**Couche entièrement connectée (fc) :**
- Type de fonction : nn.Linear

## Model AlexNet ##

In [54]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
  
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=48, kernel_size=3, padding=1)
        self.batchnorm = nn.BatchNorm2d(48)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        self.conv3 = nn.Conv2d(in_channels=48, out_channels=64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(in_channels=64, out_channels=48, kernel_size=3, padding=1)
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)

        self.fc1 = nn.Linear(in_features=3*3*48, out_features=128)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.maxpool1(x)
        x = F.relu(self.conv2(x))
        x = self.batchnorm(x)
        x = self.maxpool2(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = self.maxpool3(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def count_maxpool_operations(self, input, output, pool_layer):
        if pool_layer:
            kernel_size = pool_layer.kernel_size
            stride = pool_layer.stride
            padding = pool_layer.padding
            output_height = output.shape[2]
            output_width = output.shape[3]
            out_channels = output.shape[1]
            num_max = ((output_height + 2 * padding - kernel_size) // stride + 1) * ((output_width + 2 * padding - kernel_size) // stride + 1) * (kernel_size ** 2 - 1) * out_channels  
        else:
            num_max = 0
        return num_max

    def count_conv_operations(self, input, output, output_pooled, conv_layer, pool_layer):
        out_channels, in_channels = output.size(1), conv_layer.in_channels
        output_height, output_width = output.size(2), output.size(3)
        filter_size = conv_layer.kernel_size[0]
        stride = conv_layer.stride[0]
        padding = conv_layer.padding[0]
        num_mults = output_height * output_width * in_channels * filter_size ** 2 * out_channels
        num_adds = output_height * output_width * in_channels * filter_size ** 2 * out_channels
        num_maxs = self.count_maxpool_operations(output, output_pooled, pool_layer)
        if pool_layer:
            num_maxs = self.count_maxpool_operations(output, output_pooled, pool_layer)
        else:
            num_maxs = 0
        total_ops = num_mults + num_adds + num_maxs
        return num_mults, num_adds, num_maxs, total_ops

    def count_operations(self, x):
        conv1_out = F.relu(self.conv1(x))
        conv1_out_pooled = self.maxpool1(conv1_out)
        conv2_out = F.relu(self.conv2(conv1_out_pooled))
        conv2_out_norm = self.batchnorm(conv2_out)
        conv2_out_pooled = self.maxpool2(conv2_out_norm)
        conv3_out = F.relu(self.conv3(conv2_out_pooled))
        conv4_out = F.relu(self.conv4(conv3_out))
        conv5_out = F.relu(self.conv5(conv4_out))
        conv5_out_pooled = self.maxpool3(conv5_out)
        
        conv1_ops = self.count_conv_operations(x, conv1_out, conv1_out_pooled, self.conv1, self.maxpool1)
        conv2_ops = self.count_conv_operations(conv1_out_pooled, conv2_out, conv2_out_pooled, self.conv2, self.maxpool2)
        conv3_ops = self.count_conv_operations(conv2_out_pooled, conv3_out, conv3_out, self.conv3, None)
        conv4_ops = self.count_conv_operations(conv3_out, conv4_out, conv4_out, self.conv4, None)
        conv5_ops = self.count_conv_operations(conv4_out, conv5_out, conv5_out_pooled, self.conv5, self.maxpool3)
        
        return conv1_ops, conv2_ops, conv3_ops, conv4_ops, conv5_ops
        
    def count_fc_operations(self, input, fc_layer):
        in_features = fc_layer.in_features
        out_features = fc_layer.out_features
        num_mults = out_features * in_features
        num_adds = out_features * in_features
        num_maxs = 0
        total_ops = num_mults + num_adds
        return num_mults, num_adds, num_maxs, total_ops
    
    def count_total_operations(self, x):
        conv1_ops, conv2_ops, conv3_ops, conv4_ops, conv5_ops = self.count_operations(x)
        fc1_ops = self.count_fc_operations(x, self.fc1)
        fc2_ops = self.count_fc_operations(x, self.fc2)
        fc3_ops = self.count_fc_operations(x, self.fc3)
        total_ops = sum(op[3] for op in [conv1_ops, conv2_ops, conv3_ops, conv4_ops, conv5_ops, fc1_ops, fc2_ops, fc3_ops])
        return total_ops


**1. Le nombre de couches et sous-couches de AlexNet:**

1. **Cinq couches de convolution :**
   - `self.conv1`: Première couche de convolution avec 3 canaux d'entrée et 16 canaux de sortie, utilisant un noyau de taille 5x5.
   - `self.conv2`: Deuxième couche de convolution avec 16 canaux d'entrée et 48 canaux de sortie, utilisant un noyau de taille 3x3.
   - `self.conv3`: Troisième couche de convolution avec 48 canaux d'entrée et 64 canaux de sortie, utilisant un noyau de taille 3x3.
   - `self.conv4`: Quatrième couche de convolution avec 64 canaux d'entrée et 64 canaux de sortie, utilisant un noyau de taille 3x3.
   - `self.conv5`: Cinquième couche de convolution avec 64 canaux d'entrée et 48 canaux de sortie, utilisant un noyau de taille 3x3.

2. **Trois couches de max pooling :**
   - Trois Max pooling avec un noyau de taille 3x3 et un stride de 2.

3. **Trois couches entièrement connectées :**
   - `self.fc1`: Première couche entièrement connectée avec 432 entrées et 128 sorties.
   - `self.fc2`: Deuxième couche entièrement connectée avec 128 entrées et 128 sorties.
   - `self.fc3`: Troisième couche entièrement connectée avec 128 entrées et 10 sorties.

4. **Sept sous-couches ReLU :**
   - Utilisées après chaque opération de convolution et les deux premières couches linéaires.

In [55]:
netAlex = AlexNet().to(device)
for p in netAlex.parameters(): print(p.size())

torch.Size([16, 3, 5, 5])
torch.Size([16])
torch.Size([48, 16, 3, 3])
torch.Size([48])
torch.Size([48])
torch.Size([48])
torch.Size([64, 48, 3, 3])
torch.Size([64])
torch.Size([64, 64, 3, 3])
torch.Size([64])
torch.Size([48, 64, 3, 3])
torch.Size([48])
torch.Size([128, 432])
torch.Size([128])
torch.Size([128, 128])
torch.Size([128])
torch.Size([10, 128])
torch.Size([10])


**2. identification la taille des différents tenseurs de données Xn et de poids Wn le long du calcul:**

1. **Couche de convolution 1 (conv1) :**
   - X1: [3, 32, 32]
   - W1: [16, 3, 5, 5]
   - Bias: [16]

2. **Max pooling 1 (maxpool1) :**
   - X2: [16, 14, 14]

3. **Couche de convolution 2 (conv2) :**
   - X3: [16, 14, 14]
   - W2: [48, 16, 3, 3]
   - Bias: [48]

4. **Couche de normalisation par lots (batchnorm) :**
   - X4: [48, 14, 14]

5. **Max pooling 2 (maxpool2) :**
   - X5: [48, 6, 6]

6. **Couche de convolution 3 (conv3) :**
   - X6: [48, 6, 6]
   - W3: [64, 48, 3, 3]
   - Bias: [64]

7. **Couche de convolution 4 (conv4) :**
   - X7: [64, 6, 6]
   - W4: [64, 64, 3, 3]
   - Bias: [64]

8. **Couche de convolution 5 (conv5) :**
   - X8: [64, 6, 6]
   - W5: [48, 64, 3, 3]
   - Bias: [48]

9. **Max pooling 3 (maxpool3) :**
   - X9: [48, 2, 2]

10. **Couche entièrement connectée 1 (fc1) :**
   - X10: [1, 432]
   - W6: [128, 432]
   - Bias: [128]

11. **Couche entièrement connectée 2 (fc2) :**
   - X11: [1, 128]
   - W7: [128, 128]
   - Bias: [128]

12. **Couche entièrement connectée 3 (fc3) :**
   - X12: [1, 128]
   - W8: [10, 128]
   - Bias: [10]

**3. identification de toutes les fonctions successives (les “Fn”) avec leurs types:**

**Couche de convolution 1 (conv1) :**
- Type de fonction : Convolution - `nn.Conv2d`
- Fonction d'activation : ReLU - `nn.ReLU`
- Fonction de normalisation : `nn.BatchNorm2d`

**Couche de pooling maximal 1 (maxpool1) :**
- Type de fonction : Pooling maximal - `nn.MaxPool2d`

**Couche de convolution 2 (conv2) :**
- Type de fonction : Convolution - `nn.Conv2d`
- Fonction d'activation : ReLU - `nn.ReLU`

**Couche de normalisation par lots (batchnorm) :**
- Type de fonction : `nn.BatchNorm2d`

**Couche de pooling maximal 2 (maxpool2) :**
- Type de fonction : Pooling maximal - `nn.MaxPool2d`

**Couche de convolution 3 (conv3) :**
- Type de fonction : Convolution - `nn.Conv2d`
- Fonction d'activation : ReLU - `nn.ReLU`

**Couche de convolution 4 (conv4) :**
- Type de fonction : Convolution - `nn.Conv2d`
- Fonction d'activation : ReLU - `nn.ReLU`

**Couche de convolution 5 (conv5) :**
- Type de fonction : Convolution - `nn.Conv2d`
- Fonction d'activation : ReLU - `nn.ReLU`

**Couche de pooling maximal 3 (maxpool3) :**
- Type de fonction : Pooling maximal - `nn.MaxPool2d`

**Couche entièrement connectée 1 (fc1) :**
- Type de fonction : Entièrement connectée - `nn.Linear`
- Fonction d'activation : ReLU - `nn.ReLU`

**Couche de régularisation (dropout) :**
- Type de fonction : Régularisation par abandon - `nn.Dropout`

**Couche entièrement connectée 2 (fc2) :**
- Type de fonction : Entièrement connectée - `nn.Linear`
- Fonction d'activation : ReLU - `nn.ReLU`

**Couche entièrement connectée 3 (fc3) :**
- Type de fonction : Entièrement connectée - `nn.Linear`

## Model VGG ##

In [56]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 8 * 8, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def count_maxpool_operations(self, input, output, pool_layer, out_channels):
        kernel_maxpooling = pool_layer.kernel_size
        stride = pool_layer.stride
        padding = pool_layer.padding
        output_height = output.shape[2]
        output_width = output.shape[3]
        out_channels =  output.shape[1]
        num_max = output_height * output_width * (kernel_maxpooling**2 -1) * out_channels
        return num_max

    def count_conv_operations(self, input, output, output_pooled, conv_layer, pool_layer):
        out_channels, in_channels = output.size(1), conv_layer.in_channels
        output_height, output_width = output.size(2), output.size(3)
        filter_size = conv_layer.kernel_size[0]
        stride = conv_layer.stride[0]
        padding = conv_layer.padding[0]
        num_mults = output_height * output_width * in_channels * filter_size ** 2 * out_channels
        num_adds = output_height * output_width * in_channels * filter_size ** 2 * out_channels
        num_maxs = self.count_maxpool_operations(output, output_pooled, pool_layer, out_channels)
        total_ops = num_mults + num_adds + num_maxs
        return num_mults, num_adds, num_maxs, total_ops

    def count_operations(self, x):
        conv1_out = self.conv1(x)
        conv1_out_pooled = self.pool(F.relu(conv1_out))
        conv2_out = self.conv2(conv1_out_pooled)
        conv2_out_pooled = self.pool(F.relu(conv2_out))
        conv1_ops = self.count_conv_operations(x, conv1_out, conv1_out_pooled, self.conv1, self.pool)
        conv2_ops = self.count_conv_operations(conv1_out_pooled, conv2_out, conv2_out_pooled, self.conv2, self.pool)
        return conv1_ops, conv2_ops        

    def count_fc_operations(self, input, fc_layer):
        in_features = fc_layer.in_features
        out_features = fc_layer.out_features
        num_mults = out_features * in_features
        num_adds = out_features * in_features
        num_maxs = 0    
        total_ops = num_mults + num_adds
        return num_mults, num_adds, num_maxs, total_ops

    def count_total_operations(self, x):
        conv1_ops, conv2_ops = self.count_operations(x)
        fc1_ops = self.count_fc_operations(x, self.fc1)
        fc2_ops = self.count_fc_operations(x, self.fc2)
        fc3_ops = self.count_fc_operations(x, self.fc3)
        total_ops = sum(op[3] for op in [conv1_ops, conv2_ops, fc1_ops, fc2_ops, fc3_ops])
        return total_ops

**1. Le nombre de couches et sous-couches de VGG:**

1. **Deux couches de convolution :**
   - `self.conv1`: Première couche de convolution avec 3 canaux d'entrée et 64 canaux de sortie, utilisant un noyau de taille 3x3.
   - `self.conv2`: Deuxième couche de convolution avec 64 canaux d'entrée et 128 canaux de sortie, utilisant un noyau de taille 3x3.

2. **Deux couches de max pooling :**
   - `self.pool`: Utilisé après chaque couche de convolution pour réduire la dimensionnalité de la sortie.

3. **Trois couches entièrement connectées :**
   - `self.fc1`: Première couche entièrement connectée avec 8192 entrées (128 * 8 * 8) et 120 sorties.
   - `self.fc2`: Deuxième couche entièrement connectée avec 120 entrées et 84 sorties.
   - `self.fc3`: Troisième couche entièrement connectée avec 84 entrées et 10 sorties.
  
4. **Quarte sous-couches Relu**
   - Utilisées après chaque opération de convolution et les deux premières couches linéaires.

In [57]:
netVGG = VGG().to(device)
for p in netVGG.parameters(): print(p.size())

torch.Size([64, 3, 3, 3])
torch.Size([64])
torch.Size([128, 64, 3, 3])
torch.Size([128])
torch.Size([120, 8192])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


**2. identification la taille des différents tenseurs de données Xn et de poids Wn le long du calcul:**

1. **Couche de convolution 1 (conv1) :**
   - Données d'entrée (X0) : [3, 32, 32]
   - Poids du filtre (W1) : [64, 3, 3, 3]
   - Biais (b1) : [64]
   - Données de sortie (X1) : [64, 32, 32]

2. **Couche de max pooling 1 (pool1) :**
   - Données de sortie (X2) : [64, 16, 16]

3. **Couche de convolution 2 (conv2) :**
   - Poids du filtre (W2) : [128, 64, 3, 3]
   - Biais (b2) : [128]
   - Données de sortie (X3) : [128, 16, 16]

4. **Couche de max pooling 2 (pool2) :**
   - Données de sortie (X4) : [128, 8, 8]

5. **Aplatissement (flatten) :**
   - Données de sortie (X5) : [8192]

6. **Couche entièrement connectée 1 (fc1) :**
   - Poids du filtre (W3) : [120, 8192]
   - Biais (b3) : [120]
   - Données de sortie (X6) : [120]

7. **Couche entièrement connectée 2 (fc2) :**
   - Poids du filtre (W4) : [84, 120]
   - Biais (b4) : [84]
   - Données de sortie (X7) : [84]

8. **Couche entièrement connectée 3 (fc3) :**
   - Poids du filtre (W5) : [10, 84]
   - Biais (b5) : [10]
   - Données de sortie (X8) : [10]

**3. identification de toutes les fonctions successives (les “Fn”) avec leurs types:**

1. **Couche de convolution 1 (conv1) :**
   - Type de fonction : Convolution : nn.Conv2d
   - Fonction d'activation : F.relu

2. **Couche de max pooling 1 (pool1) :**
   - Type de fonction : nn.MaxPool2d

3. **Couche de convolution 2 (conv2) :**
   - Type de fonction : Convolution : nn.Conv2d
   - Fonction d'activation : F.relu

4. **Couche de max pooling 2 (pool2) :**
   - Type de fonction : nn.MaxPool2d

5. **Aplatissement (flatten) :**
   - Type de fonction : torch.flatten

6. **Couche entièrement connectée 1 (fc1) :**
   - Type de fonction : Couche linéaire (nn.Linear)
   - Fonction d'activation : F.relu

7. **Couche entièrement connectée 2 (fc2) :**
   - Type de fonction : Couche linéaire (nn.Linear)
   - Fonction d'activation : F.relu

8. **Couche entièrement connectée 3 (fc3) :**
   - Type de fonction : Couche linéaire (nn.Linear)

La fonction **test_model** évalue les performances d'un modèle de réseau de neurones sur un ensemble de données de test, en calculant la précision, l'erreur, le nombre total d'opérations effectuées pour une image, le temps d'évaluation et le taux d'opérations par seconde sur la durée de test.

In [58]:
def test_model(net, testloader, criterion, device):
    net.eval()
    correct = 0
    total_samples = 0 
    test_loss = 0.0
    total_test_ops = 0

    with torch.no_grad():
        start_time = time.time()
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct += (predicted == labels).sum().item()
            total_test_ops += net.count_total_operations(images)

        test_evaluate_time = time.time() - start_time

    test_accuracy = 100. * correct / total_samples
    test_loss /= len(testloader)
    test_ops_per_second = total_test_ops / test_evaluate_time

    return test_accuracy, test_loss, total_test_ops, test_evaluate_time, test_ops_per_second

La fonction **train_model** entraîne un modèle de réseau de neurones sur plusieurs époques, évalue les performances après chaque époque et avant la première, affiche l'évolution de l'erreur ou de la précision, calcule le nombre total d'opérations pour chaque époque, y compris les passes avant et arrière pour l'entraînement et une seule passe avant pour les tests, et calcule le nombre total d'opérations flottantes par seconde pendant l'entraînement.

In [59]:
def train_model(epoch_nums, net, trainloader, testloader, optimizer, criterion, device):
    total_time = 0.0
    total_ops = 0
    epoch_train_ops = 0
    
    for epoch in range(epoch_nums):
        net.train()
        sum_loss = 0.0
        correct = 0.0
        total = 0.0
        iteration_num = 0
        total_train_ops = 0
        epoch_train_time = 0.0

        if epoch > 0: 
            for i, data in enumerate(trainloader, 0):
                inputs, labels = data
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
    
                if epoch == 0 and i == 0: 
                    batch_train_ops = net.count_total_operations(inputs)
                    batch_train_ops = inputs.shape[0] * batch_train_ops
                    epoch_train_ops += batch_train_ops
    
                if torch.cuda.is_available(): 
                    torch.cuda.synchronize()
                batch_train_start = time.time()
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                sum_loss += loss.item()
    
                if torch.cuda.is_available(): 
                    torch.cuda.synchronize()
                batch_train_time = time.time() - batch_train_start
                epoch_train_time += batch_train_time
    
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                iteration_num += 1
    
                batch_train_ops = net.count_total_operations(inputs)
                batch_train_ops = inputs.shape[0] * batch_train_ops
                total_train_ops += batch_train_ops
    
            epoch_train_ops += total_train_ops

        test_accuracy, test_loss, total_test_ops, test_evaluate_time, test_ops_per_second = test_model(net, testloader, criterion, device)
        total_time += test_evaluate_time + epoch_train_time
        train_ops_per_second = epoch_train_ops / total_time
        total_ops += epoch_train_ops + total_test_ops
        
        print('[Epoch:%d] Test Acc: %.3f%% | Loss: %.3f%% | Train Ops: %d | Test Ops: %d | Time: %.6fs | Train Ops/Sec : %d ' % (
            epoch, 
            test_accuracy,
            test_loss,
            epoch_train_ops,
            total_test_ops,
            test_evaluate_time,
            train_ops_per_second
        ))

    total_params = sum(p.numel() for p in net.parameters())
    print('Time elapsed: %.3fs | Total Ops: %d | Total Parameters : %d ' % (
        total_time,
        total_ops,
        total_params,
    ))
    print('Finished Training')

### Training ###

In [60]:
net = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)    
epoch_nums = 10

train_model(epoch_nums, net, trainloader, testloader, optimizer, criterion, device)

[Epoch:0] Test Acc: 10.000% | Loss: 2.305% | Train Ops: 0 | Test Ops: 103345272 | Time: 1.215587s | Train Ops/Sec : 0 
[Epoch:1] Test Acc: 10.000% | Loss: 2.301% | Train Ops: 65408400000 | Test Ops: 103345272 | Time: 1.150398s | Train Ops/Sec : 20918008436 
[Epoch:2] Test Acc: 11.540% | Loss: 2.296% | Train Ops: 130816800000 | Test Ops: 103345272 | Time: 1.206088s | Train Ops/Sec : 25858797015 
[Epoch:3] Test Acc: 18.450% | Loss: 2.265% | Train Ops: 196225200000 | Test Ops: 103345272 | Time: 1.133450s | Train Ops/Sec : 28213423627 
[Epoch:4] Test Acc: 24.820% | Loss: 2.069% | Train Ops: 261633600000 | Test Ops: 103345272 | Time: 1.131387s | Train Ops/Sec : 29712809298 
[Epoch:5] Test Acc: 29.550% | Loss: 1.962% | Train Ops: 327042000000 | Test Ops: 103345272 | Time: 1.130169s | Train Ops/Sec : 30509397783 
[Epoch:6] Test Acc: 31.270% | Loss: 1.872% | Train Ops: 392450400000 | Test Ops: 103345272 | Time: 1.123405s | Train Ops/Sec : 31136114028 
[Epoch:7] Test Acc: 34.600% | Loss: 1.805%

In [61]:
netAlex = AlexNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(netAlex.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
epoch_nums = 10

train_model(epoch_nums, netAlex, trainloader, testloader, optimizer, criterion, device)

[Epoch:0] Test Acc: 10.000% | Loss: 2.304% | Train Ops: 0 | Test Ops: 1165701248 | Time: 1.412438s | Train Ops/Sec : 0 
[Epoch:1] Test Acc: 34.080% | Loss: 1.705% | Train Ops: 737785600000 | Test Ops: 1165701248 | Time: 1.160699s | Train Ops/Sec : 173848756845 
[Epoch:2] Test Acc: 42.820% | Loss: 1.490% | Train Ops: 1475571200000 | Test Ops: 1165701248 | Time: 1.125994s | Train Ops/Sec : 209727245765 
[Epoch:3] Test Acc: 53.170% | Loss: 1.284% | Train Ops: 2213356800000 | Test Ops: 1165701248 | Time: 1.218655s | Train Ops/Sec : 220868966199 
[Epoch:4] Test Acc: 60.410% | Loss: 1.107% | Train Ops: 2951142400000 | Test Ops: 1165701248 | Time: 1.131044s | Train Ops/Sec : 230384591550 
[Epoch:5] Test Acc: 63.680% | Loss: 1.037% | Train Ops: 3688928000000 | Test Ops: 1165701248 | Time: 1.149699s | Train Ops/Sec : 235204726598 
[Epoch:6] Test Acc: 67.230% | Loss: 0.960% | Train Ops: 4426713600000 | Test Ops: 1165701248 | Time: 1.243838s | Train Ops/Sec : 237843428925 
[Epoch:7] Test Acc: 67.

In [62]:
netVGG = VGG().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(netVGG.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
epoch_nums = 10

train_model(epoch_nums, netVGG, trainloader, testloader, optimizer, criterion, device)

[Epoch:0] Test Acc: 9.520% | Loss: 2.304% | Train Ops: 0 | Test Ops: 3424596912 | Time: 1.183321s | Train Ops/Sec : 0 
[Epoch:1] Test Acc: 44.430% | Loss: 1.540% | Train Ops: 2167466400000 | Test Ops: 3424596912 | Time: 1.373083s | Train Ops/Sec : 476998659640 
[Epoch:2] Test Acc: 53.390% | Loss: 1.283% | Train Ops: 4334932800000 | Test Ops: 3424596912 | Time: 1.137444s | Train Ops/Sec : 565688446328 
[Epoch:3] Test Acc: 60.660% | Loss: 1.116% | Train Ops: 6502399200000 | Test Ops: 3424596912 | Time: 1.191456s | Train Ops/Sec : 599838397616 
[Epoch:4] Test Acc: 63.650% | Loss: 1.020% | Train Ops: 8669865600000 | Test Ops: 3424596912 | Time: 1.163049s | Train Ops/Sec : 619864624199 
[Epoch:5] Test Acc: 64.870% | Loss: 0.983% | Train Ops: 10837332000000 | Test Ops: 3424596912 | Time: 1.589778s | Train Ops/Sec : 617010510324 
[Epoch:6] Test Acc: 69.120% | Loss: 0.872% | Train Ops: 13004798400000 | Test Ops: 3424596912 | Time: 1.135618s | Train Ops/Sec : 628838530556 
[Epoch:7] Test Acc: 7

In [63]:
netRes = ResNet18().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(netRes.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
epoch_nums = 10

train_model(epoch_nums, netRes, trainloader, testloader, optimizer, criterion, device)

[Epoch:0] Test Acc: 8.830% | Loss: 2.307% | Train Ops: 0 | Test Ops: 6253260800 | Time: 1.790318s | Train Ops/Sec : 0 
[Epoch:1] Test Acc: 56.440% | Loss: 1.224% | Train Ops: 3957760000000 | Test Ops: 6253260800 | Time: 1.767439s | Train Ops/Sec : 276940073380 
[Epoch:2] Test Acc: 62.830% | Loss: 1.045% | Train Ops: 7915520000000 | Test Ops: 6253260800 | Time: 1.782650s | Train Ops/Sec : 295267498390 
[Epoch:3] Test Acc: 69.030% | Loss: 0.889% | Train Ops: 11873280000000 | Test Ops: 6253260800 | Time: 1.776960s | Train Ops/Sec : 301976290765 
[Epoch:4] Test Acc: 65.380% | Loss: 1.042% | Train Ops: 15831040000000 | Test Ops: 6253260800 | Time: 1.790285s | Train Ops/Sec : 305354740214 
[Epoch:5] Test Acc: 69.640% | Loss: 0.872% | Train Ops: 19788800000000 | Test Ops: 6253260800 | Time: 1.764334s | Train Ops/Sec : 307538372201 
[Epoch:6] Test Acc: 75.050% | Loss: 0.722% | Train Ops: 23746560000000 | Test Ops: 6253260800 | Time: 1.786782s | Train Ops/Sec : 308922587753 
[Epoch:7] Test Acc: